In [3]:
# F1 賽事對城市銷售額的影響分析：延伸專案

本 Notebook 旨在完整呈現「F1 賽事對城市旅遊銷售額影響分析」專案的分析流程。我們將從資料載入、清理、整合開始，逐步進行核心分析，最終建立一個預測模型來量化各因素對銷售額的影響，並提出商業建議。

---

### 分析步驟：
1.  **資料載入**：載入所有相關的 CSV 檔案。
2.  **資料清理與整合**：將多個數據集合併成一個用於分析的 DataFrame。
3.  **核心分析**：進行描述性統計和 t-檢定，比較賽事週與非賽事週的差異。
4.  **模型建立**：使用多元線性迴歸模型來預測銷售額。

# 載入所需的函式庫
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import re

print("函式庫載入成功！")

SyntaxError: invalid character '「' (U+300C) (867600647.py, line 3)

In [4]:
### 1. 資料載入與初步檢視

我們將載入所有原始 CSV 檔案，並檢查它們的結構。
* **注意**：`../` 表示回到上層目錄，這是為了正確讀取 `data/raw` 資料夾中的檔案。

SyntaxError: invalid character '，' (U+FF0C) (198597331.py, line 3)

In [5]:
# 載入所有 CSV 檔案
try:
    df_sales = pd.read_csv('../data/raw/f1_weekly_sales_by_city拷貝.csv')
    df_trends = pd.read_csv('../data/raw/google_trends_long_format.csv')
    df_products = pd.read_csv('../data/raw/f1_event_products_template拷貝.csv')
    df_dts = pd.read_csv('../data/raw/f1_dts_by_country_filled.csv')
    print("所有檔案載入成功！")
except FileNotFoundError as e:
    print(f"錯誤：找不到檔案，請確認檔案路徑是否正確。錯誤資訊：{e}")
    exit()

# 顯示 df_sales 的前五行，方便檢查
print("\n--- 銷售數據 (df_sales) 前五行 ---")
print(df_sales.head())

NameError: name 'pd' is not defined

In [6]:
### 2. 資料清理與整合

為了將所有數據整合在一起，我們需要進行以下步驟：
- 將日期欄位轉換為統一的格式。
- 清理 `f1_event_products_template` 中的城市名稱。
- 根據日期和城市進行數據合併。

SyntaxError: invalid character '，' (U+FF0C) (220641844.py, line 3)

In [7]:
# -----------------------------------------------------------------------------
# 資料清理與前置處理
# -----------------------------------------------------------------------------

# 將日期欄位轉換為日期時間格式
df_sales['週'] = pd.to_datetime(df_sales['週'])
df_trends['date'] = pd.to_datetime(df_trends['date'])
df_dts['country'] = df_dts['country'].str.strip()

# 處理 df_products 的 race_location 欄位，提取城市/國家名稱
def clean_location(location):
    match = re.match(r'(.+?)\s?\(', location)
    if match:
        return match.group(1).strip()
    return location

df_products['race_location_cleaned'] = df_products['race_location'].apply(clean_location)

# -----------------------------------------------------------------------------
# 資料合併
# -----------------------------------------------------------------------------

# 1) 彙總產品和評論數據
df_products_agg = df_products.groupby('race_location_cleaned').agg(
    產品數量=('product_count', 'sum'),
    評論數=('review_count', 'sum')
).reset_index()

# 2) 合併銷售額與 Google 趨勢數據
df_trends_f1 = df_trends[df_trends['keyword'] == 'F1'].copy()
df_trends_f1.rename(columns={'date': '週', 'trend_score': 'F1_趨勢分數'}, inplace=True)
df_merged = pd.merge(df_sales, df_trends_f1[['週', 'F1_趨勢分數']], on='週', how='left')

# 3) 合併產品數據
city_name_mapping = {
    'Baku': 'Baku', 'Singapore': 'Singapore', 'Suzuka': 'Japan',
    'Shanghai': 'China', 'Las Vegas': 'America', 'Miami': 'America'
}
df_merged['city_for_merge'] = df_merged['城市'].map(city_name_mapping).fillna(df_merged['城市'])
df_merged = pd.merge(df_merged, df_products_agg, left_on='city_for_merge', right_on='race_location_cleaned', how='left')
df_merged.drop(columns=['city_for_merge', 'race_location_cleaned'], inplace=True)

# 4) 填補合併後可能產生的 NaN 值
df_merged['產品數量'].fillna(0, inplace=True)
df_merged['評論數'].fillna(0, inplace=True)
df_merged['F1_趨勢分數'].fillna(df_merged['F1_趨勢分數'].mean(), inplace=True)

print("資料合併完成！合併後數據的前五行：")
print(df_merged.head())

NameError: name 'pd' is not defined

In [8]:
### 3. 核心分析：賽事週與非賽事週銷售額比較 (t-檢定)

我們將賽事週的銷售額與非賽事週的銷售額進行比較，以驗證賽事對銷售的影響是否顯著。

SyntaxError: invalid character '，' (U+FF0C) (1810935760.py, line 3)

In [9]:
sales_race_week = df_merged[df_merged['是否比賽週'] == True]['銷售額']
sales_non_race_week = df_merged[df_merged['是否比賽週'] == False]['銷售額']

t_stat, p_value_ttest = stats.ttest_ind(sales_race_week, sales_non_race_week, equal_var=False, nan_policy='omit')

print("賽事週與非賽事週銷售額比較 (t-檢定)")
print(f"賽事週平均銷售額: ${sales_race_week.mean():.2f}")
print(f"非賽事週平均銷售額: ${sales_non_race_week.mean():.2f}")
print(f"p-value: {p_value_ttest:.4f}")

if p_value_ttest < 0.05:
    print("結論: 賽事週的銷售額顯著高於非賽事週。")
else:
    print("結論: 賽事週與非賽事週的銷售額沒有顯著差異。")

NameError: name 'df_merged' is not defined

In [10]:
### 4. 建立多元線性迴歸模型

建立一個多元線性迴歸模型來量化各變數對銷售額的影響。

SyntaxError: invalid character '。' (U+3002) (217478634.py, line 3)

In [11]:
# 定義自變數 (X) 和應變數 (y)
X = df_merged[['是否比賽週', 'F1_趨勢分數', '產品數量', '評論數']].copy()
y = df_merged['銷售額']

# 將布林值的 '是否比賽週' 轉換為 0/1
X['是否比賽週'] = X['是否比賽週'].astype(int)

# 增加截距項
X = sm.add_constant(X)

# 建立並擬合 OLS 模型
model = sm.OLS(y, X, missing='drop').fit()

# 輸出模型摘要
print("多元線性迴歸模型結果")
print(model.summary())

NameError: name 'df_merged' is not defined